In [ ]:
#dataset
class Wav_Dataset(Dataset):
    def __init__(self,root):
        '''root   transform_state  transform  '''
        self.wav_files = [x.path for x in os.scandir(root) if x.name.endswith(".wav")]
        self.wav_files.sort(key = lambda x : int(x.replace(root+"...","").split('.')[0]))
        self.wav_files = np.array(self.wav_files)
        #print(self.wav_files)
        
    def __getitem__(self, index):
        #waveform  sample_rate  
        waveform, sample_rate = torchaudio.load_wav(self.wav_files[index])
        #waveform = torch.unsqueeze(waveform,1)
        #print(waveform.shape)
        tensor_minusmean = waveform - (waveform *1.0).mean()
        waveform = tensor_minusmean/tensor_minusmean.abs().max()
        # Spectorgarm = transforms.Spectrogram()(waveform)   
        #if waveform.shape[1]<64000:
        #waveform = waveform.permute(1,0)
        #print(waveform.shape)
        return waveform,sample_rate
    def __len__(self):
        return len(self.wav_files)

In [ ]:
class Wav_test(Dataset):
    def __init__(self,root):
        '''root transform_state  transform  '''
        self.wav_files = [x.path for x in os.scandir(root) if x.name.endswith(".wav")]
        self.wav_files.sort(key = lambda x : int(x.replace(root,"").split('.')[0]))
        self.wav_files = np.array(self.wav_files)
        #print(self.wav_files[:10])
      
    def __getitem__(self, index):
        waveform, sample_rate = torchaudio.load(self.wav_files[index])
        #tensor_minusmean = waveform - waveform.mean()
        #waveform = tensor_minusmean/tensor_minusmean.abs().max()
        #Spectorgarm = transforms.Spectrogram()(waveform) 
        #if waveform.shape[1]<64000:
        #waveform = waveform.permute(1,0)
        return waveform,sample_rate
    def __len__(self):
        return len(self.wav_files)
       

In [ ]:
wav_data_path = '/...../'
#读取音频数据
my_wavdataset = Wav_Dataset(wav_data_path)
my_wavloader = DataLoader(my_wavdataset,batch_size=BATCH_SIZE,num_workers=0,shuffle=False)

test_data = Wav_test('/..../')
test_dataloader = DataLoader(test_data,batch_size=BATCH_SIZE,num_workers=0,shuffle=False)

In [ ]:

class My_Img_Dataset(Dataset):
    def __init__(self, root, transform=None):
        self.image_files = [x.path for x in os.scandir(root) if x.name.endswith(".jpg") or x.name.endswith(".png") or x.name.endswith(".JPG")]
        self.image_files.sort(key = lambda x : int(x.replace(root +'...','').split('.')[0]))
        self.image_files = np.array(self.image_files)
        #print(self.image_files[:10])
        self.transform= transform
    def __getitem__(self, index):
        if self.transform:
            image = Image.open(self.image_files[index]).convert('RGB')
            image = self.transform(image)  
            return image
        else:
            return Image.open(self.image_files[index]).convert('RGB')

    def __len__(self):
        return len(self.image_files)

In [ ]:
class My_Img_G_Dataset(Dataset):
    def __init__(self, root,batch_size,audio_len,transform=None):
        self.batch_size = batch_size+1
        self.image_files = [x.path for x in os.scandir(root) if x.name.endswith(".jpg") or x.name.endswith(".png") or x.name.endswith(".JPG")]
        self.image_files.sort(key = lambda x : int(x.replace(root+"....",'').split('.')[0]))
        self.audio_len = audio_len
        self.image_files = [self.image_files[i] for i in range(0,len(self.image_files),self.batch_size)]
        self.image_files_gj = []
        if len(self.image_files) == 1:
            x = self.image_files[0]
            for k in range(audio_len):
                self.image_files_gj.append(x)
        else:  
            for i in range(1,len(self.image_files)):
                x = self.image_files[i]
                for k in range(batch_size):
                    self.image_files_gj.append(x)
        self.image_files = self.image_files_gj
        self.image_files = np.array(self.image_files)
        #print(self.image_files[:10])
        self.transform= transform
    def __getitem__(self, index):
        if self.transform:
            image = Image.open(self.image_files[index]).convert('RGB')
            image = self.transform(image)  
            #print(image.shape)
            return image
        else:
            return Image.open(self.image_files[index]).convert('RGB')

    def __len__(self):
        return len(self.image_files)

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((200,200)),
    torchvision.transforms.CenterCrop(200),
    torchvision.transforms.ToTensor(), 
    torchvision.transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
                                           ])

transform1 = torchvision.transforms.Compose([
    torchvision.transforms.Resize((200,200)),
    torchvision.transforms.CenterCrop(200),
    torchvision.transforms.ToTensor(),  
    torchvision.transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])  
])
#img_data_path = '/.../'
img_data_path = '/.../'
img_D_path = '/.../'
#img_D_path = '/.../'
my_imgdataset = My_Img_Dataset(img_D_path,transform)
my_imgloader = DataLoader(my_imgdataset,batch_size=BATCH_SIZE,num_workers=0,shuffle=False)

my_imgGdataset = My_Img_G_Dataset('/../',BATCH_SIZE,len(my_wavdataset),transform)
my_imgGloader = DataLoader(my_imgGdataset,batch_size=BATCH_SIZE,num_workers=0,shuffle=False)
my_imgG1dataset = My_Img_G_Dataset('/.../',BATCH_SIZE,len(test_data),transform1)
my_imgG1loader = DataLoader(my_imgG1dataset,batch_size=BATCH_SIZE,num_workers=0,shuffle=False)
#print(my_imgG1loader)
#q = np.array(my_imgG1loader)
#print(q.shape)
